### **WELCOME TO StockPredDemo_v.6.0.**

# AI-aid Investment analysis WebApp v6.0
## Ethan Aug***usto Gonzalez Bonorino***
### Status report for v5.0

- Build Deep Learning models with TensorFlow. (2 out of 4)
- Improve front end design. ☑
- Full implementation of Portfolio Optimization
- plotly express has been fixed
- Added technical indicators

### Things to fix/add for v6.0

- Finish models
- Finish adding technical indicators
- If twint is fixed merge sentiment and historical data for training
- ALTERNATIVE: use finViz or other package to scrape financial news and use news sentiment instead of tweets.
- Deploy WebApp for demo and sharing

# **NOTE: New sentiment model runs faster (100 tweets < 1m) but it's constrained to tweets in English. Just to consider in the report. Also, we run out of tweets to scrape after fetching 5 per day for a year and change.**

## **Technologies** 📋

* **Twint** -> Tweets scraper
* **Streamlit** -> To build and share our WebApp
* **Python** -> Programming language of choice
* **Tensorflow** -> For optimizing and implementing models
* **Yfinance** -> For historical and technical stock data
* **BERT/VADER** -> To compute tweets sentiment scores

## Install and Import dependencies

## Twint

In [1]:
# Twint

!git clone --depth=1 https://github.com/twintproject/twint.git 

Cloning into 'twint'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 3), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.


## Open twint -> requirements.txt and specify aiohttp==3.7.0 before running the next cell

In [2]:
## must update requirements.txt for compatibility aiohttp==3.7.0

!cd /content/twint && pip3 install . -r requirements.txt --quiet

# Yahoo Finance API

!pip install yfinance --upgrade --no-cache-dir

## Pyngrok

!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet

!pip install transformers --quiet

!pip install tensorflow --quiet

!pip install numpy==1.19.3 --quiet

!pip install emojis --quiet

!pip install PyPortfolioOpt --quiet

!pip install pyyaml==5.4.1 --quiet

!pip install pandas_ta --quiet

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 1.3 MB 35.8 MB/s 
     |████████████████████████████████| 263 kB 54.8 MB/s 
     |████████████████████████████████| 378 kB 46.1 MB/s 
     |████████████████████████████████| 271 kB 66.9 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 291 kB 67.1 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 138 kB 37.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages t

### Now, run the following cell.

In [3]:
from pyngrok import ngrok

import nest_asyncio
nest_asyncio.apply()

## Streamlit WebApp design

In [15]:
%%writefile app.py
import twint
import time
import pandas as pd
import pandas_ta as ta
import numpy as np

import pandas_datareader as pdr

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

import os
import re
import emojis
from PIL import Image

import streamlit as st
import yfinance as yf

# from pypfopt.expected_returns import mean_historical_return, ema_historical_return
# from pypfopt.risk_models import CovarianceShrinkage, sample_cov, exp_cov
# from pypfopt.efficient_frontier import EfficientFrontier
# from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
# from pypfopt import HRPOpt
# from pypfopt.efficient_frontier import EfficientCVaR

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Flatten, Dropout
from keras.utils.vis_utils import plot_model

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from scipy.special import softmax


import datetime

from dateutil import parser

timestr = time.strftime("%Y%m%d")

######## FUNCTIONS ########

def get_ticker(name):

    ticker = yf.Ticker(name)  
    return ticker

def get_hist_data(ticker, start, end, include_period = False, period = 'max'):

    res = 0

    if include_period == True:
        ticker = yf.Ticker(ticker)
        dataset = ticker.history(period=period)
        res = dataset

    else:
        res = yf.download(ticker, start=start, end=end)

    return pd.DataFrame(res)

def compute_RSI(df, window, adjust=False):
 
  delta = df['y']
  loss = delta.copy()
  gains = delta.copy()

  loss[loss > 0] = 0
  loss[loss == 0] = 1

  gain_ewm = gains.ewm(com=window - 1, adjust=adjust).mean()
  loss_ewm = abs(loss.ewm(com=window - 1, adjust=adjust).mean())

  RS = gain_ewm / loss_ewm
  RSI = 100 - 100 / (1 + RS)

  return RSI

def append_TI(df, tech_indicators):

  for indicator in tech_indicators:
    if indicator == "SMA2 (rolling mean)":
      df["SMA_2"] = df.Close.rolling(2).mean()

    if indicator == "RSI":
      df['RSI'] = compute_RSI(df, 14)
    
    if indicator == "Force Index":
      df["Force_Index"] = df["Close"] * df["Volume"]

    if indicator == "MACD":
      df.ta.macd(close='close', fast=12, slow=26, signal=9, append=True)

    if indicator == "Aroon Oscillator":
      df.ta.aroon(append = True)
      df = df.drop(["AROOND_14", "AROONU_14"], axis = 1)

    if indicator == "On-Balance Volume (OBV)":
      df.ta.obv(append = True)

  return df

def get_recommendations(ticker):

    analyst = ticker.recommendations

    analyst_df = pd.DataFrame(analyst)

    return analyst_df

# News feed
def print_news(ticker):

    news = ticker.news

    news_df = pd.DataFrame()
    titles = []
    publishers = []
    links = []

    for doc in news:

      title = doc['title']
      publisher = doc['publisher']
      link = doc['link']

      titles.append(title)
      publishers.append(publisher)
      links.append(link)

    news_df['Title'] = titles
    news_df['Publisher'] = publishers
    news_df['Link'] = links

    return news_df

def plot_time_series(df):

    fig = go.Figure([go.Scatter(x=df.index, y=df['Adj Close'])])

    fig.update_layout(
        title= str(ticker_name) + ' data',
        yaxis_title= str(ticker_name) + ' Stock price')

    return st.plotly_chart(fig, use_container_width=True)

def make_candlestick(company, ticker_name):
    fig = go.Figure(data=[go.Candlestick(
                x=company.index,
                open=company['Open'],
                high=company['High'],
                low=company['Low'],
                close=company['Close'])])

    fig.update_layout(
        title= str(ticker_name) + ' data',
        yaxis_title= str(ticker_name) + ' Stock price')
    
    return st.plotly_chart(fig, use_container_width=True)

def make_ohlc(company, ticker_name):
    fig = go.Figure(data=[go.Ohlc(
                x=company.index,
                open=company['Open'],
                high=company['High'],
                low=company['Low'],
                close=company['Close'])])

    fig.update_layout(
        title= str(ticker_name) + ' data',
        yaxis_title= str(ticker_name) + ' Stock price')
    
    return st.plotly_chart(fig, use_container_width=True)

def make_efficient_frontier(x, y):
    fig = go.Figure(data = [go.Scatter(x=x, y=y, mode = 'markers', marker=dict(
                            size=7,
                            color=y, #set color equal to a variable
                            colorscale='Viridis', # one of plotly colorscales
                            showscale=False))])

    fig.update_layout(
        title= "Portfolio Efficient Frontier",
        xaxis_title = "Risk (Volatility)",
        yaxis_title= "Expected returns")
    
    return st.plotly_chart(fig, use_container_width=True)

def make_efficient_frontier_opt(x, y, min_port_x, min_port_y, max_port_x, max_port_y, min_volatility = True):

    fig = go.Figure(data = [go.Scatter(x=x, y=y, mode = 'markers', marker=dict(
                            size=7,
                            color=y, #set color equal to a variable
                            colorscale='Viridis', # one of plotly colorscales
                            showscale=False))])

    fig.update_layout(
        title= "Optimized Portfolio Allocation in Efficient Frontier",
        xaxis_title = "Risk (Volatility)",
        yaxis_title= "Expected returns")
    
    if min_volatility == True:

        fig.add_trace(go.Scatter(x=[min_port_x], y=[min_port_y],
                        mode='markers', marker_symbol = 'star', marker = dict(color='blue', size = 13),
                        name='Minimum Volatility Portfolio'))
        
    else:
        fig.add_trace(go.Scatter(x=[max_port_x], y=[max_port_y],
                        mode='markers', marker_symbol = 'star', marker = dict(color='red', size = 13),
                        name='Maximum Sharpe Ratio Portfolio'))

    
    return st.plotly_chart(fig, use_container_width=True)

def portfolio_metrics(returns, weights, rf, index='Trial'):
    
    '''
    This function generates the relative performance metrics that will be reported and will be used
    to find the optimal weights.
    
    Parameters:
    weights: initialized weights or optimal weights for performance reporting
    
    '''   
    
    rp = (returns.mean()*252)@weights 
    port_var = weights@(cov*252)@weights
    sharpe = (rp-rf)/np.sqrt(port_var)
    df = pd.DataFrame({"Expected Return": rp,
                       "Portfolio Variance":port_var,
                       'Portfolio Std': np.sqrt(port_var),
                       'Sharpe Ratio': sharpe}, index=[index])
    return df

def optimal_portfolio_weights(weights, stocks):
    df = pd.DataFrame({"Stock": stocks,
                       "Weights": weights})
    df.Weights = df.Weights.map(lambda x: '{:.2%}'.format(x))
    return df

def get_tweets(ticker, num_tweets, start, end):

    config = twint.Config()

    config.Search = "$" + ticker
    config.Lang = "en"
    config.Since = start
    config.Until = end
    config.Limit = num_tweets
    #config.Min_retweets = 50

    config.Pandas = True

    twint.run.Search(config)

    tweets_df = twint.storage.panda.Tweets_df.head(num_tweets)

    if len(tweets_df) < 1 or type(tweets_df['date']) == float:
        data = pd.DataFrame([["None", start]], columns=['tweet', 'date'])
    else:
        data = pd.DataFrame(tweets_df[['tweet', 'date']])

    return data

def tweetdf_to_list(tweet_df):
    tweets = [line for line in tweet_df['tweet']]

    return tweets

def tweetDates_to_DateTime(df):
  i = 0
  for column in df['Date']:
    dt_obj = datetime.datetime.strptime(column, '%Y-%m-%d %H:%M:%S')
    i = i + 1

  return df

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def compute_sentiment(model_name):
 
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    config = AutoConfig.from_pretrained(model_name)

    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

    return classifier

def convert_ISO8601(df):
  i = 0
  for column in df['Date']:
    date = pd.to_datetime(column)
    date = date.date()
    df['Date'][i] = date
    i = i + 1
  
  return df

def correlation_matrix(df):

  df_corr = df.corr() # Generate correlation matrix

  fig = go.Figure()
  fig.add_trace(
      go.Heatmap(
          x = df_corr.columns,
          y = df_corr.index,
          z = np.array(df_corr)
      )
  )

  return st.plotly_chart(fig, use_container_width=True)

######## MAIN BODY ########

st.title('AI-aid investment analysis tool')
st.subheader("by Augusto Gonzalez-Bonorino and Ethan Aug")

# Containers help separate content. 
# Make sure to assign relevant names to distinguish containers
description = st.container()
description.write("""The objective of this Webapp is to provide informative data and models of the stock market while empowering the user.
                    This app provides a lot of flexibility in terms of input: Choose your own ticker and time range to investigate and gain insight
                    on key metrics. With the everincreasing influence of technology, we felt it was important to include a social media sentiment score.
                    Scraping Twitter in real time, the app outputs tweets containing your selected ticker, calculates a daily sentiment score and uses
                    this alongside historical data in the models. There is also flexibility in model selection and hyperparameter tuning. 
                    Investigate different stocks, dates, models and hyperparamters to come to a data driven solution. Power to the user. """)

######### SIDEBAR MENU ########
st.sidebar.header("Data Science Capping \n Financial Engineering")

######### MAIN BODY ##########
ticker_name = st.text_input("Enter ticker to analyze")

ticker = get_ticker(ticker_name)

st.text('Specify range of time:')

today = datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)

start_date = st.date_input('Start date', today)
end_date = st.date_input('End date', tomorrow)

if len(ticker_name) >= 1: 
  stock_prices = get_hist_data(ticker_name, start_date, end_date)


######## COMPANY SUMMARY ########

# detailed summary on Ticker
if st.checkbox('Show company summary'):

    st.subheader('Company summary on: ' + str(ticker.info['longName']))
    st.write(ticker.info['longBusinessSummary']) 

######## RAW DATA DATAFRAME ########

## Make options inside checkbox bubbles a dropdown menu instead of more checkbox objects.

# Raw data for Ticker
if st.checkbox('Show raw data'):

    if len(ticker_name) < 1:
        st.warning("Please enter a ticker to analyze")

    if start_date < end_date:
        st.success('Start date: `%s`\n\nEnd date: `%s`' % (start_date, end_date))
    else:
        st.error('Error: End date must fall after start date.')

    st.subheader('Raw data from ' + str(start_date) + ' to ' + str(end_date))

    # fetches the data: Open, Close, High, Low and Volume
    if len(ticker_name) >= 1:
      company = get_hist_data(ticker_name, start_date, end_date)

      st.write(company)

    # Add fundamental indicators from ticker.info

    with st.form("form2"):
        if st.checkbox("Analyze fundamentals"):

            indicators = st.multiselect(
            'Select from the following list',
            ['ebitda', 'twoHundredDayAverage', ])

            # This should be able to handle a list of indicators
            for indicator in indicators:
                
                st.write(str(indicator))
                st.write(ticker.info[indicator])

        # News NOTE: GIVES ERRORS
        if st.checkbox("Show " + str(ticker.info['longName']) + " related news (BETA)"):
            news = print_news(ticker)
            st.write(news)

        # analyst recommendations
        if st.checkbox("Analyst recommendations"):
            recoms = get_recommendations(ticker)
            st.write(recoms)

        submitted = st.form_submit_button("Submit")
        if submitted:
            st.success("Loading data!")
 
######## CHARTS ########

# plots the graph
if st.checkbox('Show charts for Start date: %s\n\nEnd date: %s' % (start_date, end_date)):

    ts_start_date = st.date_input('Time series start date', start_date)
    ts_end_date = st.date_input('Time series end date', end_date)

    ts_stock_prices = get_hist_data(ticker_name, ts_start_date, ts_end_date)

    plot_choice = st.selectbox("Choose a plot to display", options = ["Time Series", "Candlestick", "OHLC"])

    if plot_choice == "Time Series":

        st.subheader('Time series plot for ' + str(ticker.info['longName']) + ' price')    
        plot_time_series(ts_stock_prices)
    
    elif plot_choice == "Candlestick":

        st.subheader('Candlestick chart for ' + str(ticker.info['longName']) + ' price')
        make_candlestick(ts_stock_prices, ticker_name)

    elif plot_choice == "OHLC":

        st.subheader('OHLC chart for ' + str(ticker.info['longName']) + ' price')
        make_ohlc(ts_stock_prices, ticker_name)
  
######## TWEETS ########

# input bars to select ticker and num_tweets

if st.checkbox("Build Sentiment Analysis Pipeline..."):

      rangeDays = (end_date - start_date).days

      text = st.text_input('Enter text to analyze')

      # Here we can have an input box for the user to select a model
      model_name = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

      # Load models
      with st.spinner("Loading..."):

          classifier = compute_sentiment(model_name)
      
      st.success("Ready!")

      st.write("Here are our predictions. NOTE: Daily sentimet is computed as a weighted sum of the number of positive and negative tweets." +
      "If the sum is positive then daily sentiment is considered overall positive, negative otherwise.")

      results = classifier(text)

      labels = []
      scores = []

      for i in results:
          label = i['label']
          score = i['score']

          if label == "Neutral": 
              labels.append(label+" 0")
          elif label == "Positive":
              labels.append(label+" 1")
          else:
              labels.append(label+"-1")     
          
          scores.append(score)

      results_df = pd.DataFrame(results)

      sent_scores = []

      for i in range(len(labels)):
          sent_score = float(labels[i][-2:]) * scores[i]
          sent_scores.append(sent_score)

      scores_df = pd.concat([pd.Series(text), results_df], axis=1)

      scores_df["sent_score"] = sent_scores

      st.write(scores_df)

      daily_avg_sentiment = ( sum(scores_df['sent_score']) / rangeDays )

      ## Derive more efficient way of computing sentiment score ##
      st.write("Daily Average Sentiment")
      st.write(daily_avg_sentiment)

      if daily_avg_sentiment < 0:
          st.write(emojis.encode("NEGATIVE :chart_with_downwards_trend:"))

      else:
          st.write(emojis.encode("POSITIVE :chart_with_upwards_trend:"))


######### DATASET ##########

if st.checkbox("Build and Display Dataset"):

    if len(ticker_name) < 1:
        st.warning("Please enter a ticker to analyze")

    st.success('Loading full dataset for ' + str(ticker.info['longName']))

    with st.spinner("Loading..."):
        # fetches the data: Open, Close, High, Low and Volume
        companyDataset = get_hist_data(ticker_name, start_date, end_date, include_period=True)

        date_df2 = companyDataset.reset_index()
        firstDay2 = date_df2['Date'][0].date()
        lastDay2 = date_df2['Date'][len(date_df2['Date'])-1].date()

        start = time.time()
        num_tweets2 = 2

        rangeDays2 = (lastDay2 - firstDay2).days
        st.write("You selected ", num_tweets2, " tweets per day, for ", rangeDays2, " days.")

        tweets_train = pd.DataFrame()
        tweets_train_TEMP = pd.DataFrame()

        for d in range(rangeDays2-1):

            st.write("DAY: ", firstDay2)

            dailyEnd2 = firstDay2 + datetime.timedelta(days=1)

            tweets_with_date_day2 = get_tweets(ticker_name, num_tweets2, firstDay2.strftime("%Y-%m-%d %H:%M:%S"), str(dailyEnd2))

            tweets_day2 = tweetdf_to_list(tweets_with_date_day2)
            
            if len(tweets_day2) > 1:

                cleanTweets_day2 = [preprocess(tweet) for tweet in tweets_day2]

                tweets_train_TEMP['Date'] = tweets_with_date_day2['date']

                tweets_train_TEMP['Tweet'] = pd.Series(cleanTweets_day2)

                tweets_train_TEMP = tweetDates_to_DateTime(tweets_train_TEMP)

                tweets_train = tweets_train.append(tweets_train_TEMP, ignore_index = True)
                
            firstDay2 += datetime.timedelta(days=1)

        st.write(tweets_train['Tweet'].tolist())

    model_name = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

    # Load models
    with st.spinner("Loading..."):

        start1 = time.time()

        classifier = compute_sentiment(model_name)
    
        st.success("Classifier ready! Building dataset...")

        results_train = classifier(tweets_train['Tweet'].tolist())

        labels_train = []
        scores_train = []

        for i in results:
            label = i['label']
            score = i['score']

            if label == "Neutral": 
                labels_train.append(label+" 0")
            elif label == "Positive":
                labels_train.append(label+" 1")
            else:
                labels_train.append(label+"-1")     
            
            scores_train.append(score)

        results_train = pd.DataFrame(results_train)

        sent_scores_train = []

        for i in range(len(labels_train)):
            sent_score = float(labels[i][-2:]) * scores[i]
            sent_scores_train.append(sent_score)

        scores_df_train = pd.concat([tweets_train, results_train], axis=1)

        scores_df_train["sent_score"] = sent_scores_train

        st.write(scores_df_train)

        #Group by date and sum sent scores
        daily_sent_df = scores_df.groupby( [ pd.to_datetime( scores_df_train['Date'] ).dt.date ] )['sent_score'].sum()

        daily_sent_df = pd.DataFrame(daily_sent_df)

        #make date a column so we can edit it
        companyDataset = companyDataset.reset_index()

        #change date column (timestamp) to date
        companyDataset = convert_ISO8601(companyDataset)

        #set date back to index
        companyDataset = companyDataset.set_index('Date')

        #join historical data with daily sent avg data on index (date)
        dataset = companyDataset.join(daily_sent_df)

        end1 = time.time()

    st.write("It took ", round(end1 - start1, 2), " seconds to load models and build dataset of dimension ", dataset.shape )
    st.write(dataset)

    # Plot correlation matrix

    st.subheader("Correlation Matrix")
    correlation_matrix(dataset)

######### MODELS ##########

# https://github.com/BenjiKCF/Neural-Net-with-Financial-Time-Series-Data/blob/master/5.%20Recurrent%20Neural%20Network.ipynb

if st.checkbox("Machine Learning"):

  options = ['XGBoost', "Recurrent Neural Network (RNN)", "Random Forest Classifier", "LSTM"]

  model_name = st.selectbox("Choose a model", options)

  st.write("You chose: ", model_name)

  if st.checkbox("Build training dataset for " + ticker_name):

    from_date = st.date_input('From', today)
    to_date = st.date_input('To', tomorrow)

    if len(ticker_name) >= 1: 
      train_stock_prices = get_hist_data(ticker_name, from_date, to_date)

    st.write("Here is your dataset for training:")
    st.write(train_stock_prices)

    tech_indicator_options = ['RSI', 'SMA2 (rolling mean)', 'Force Index', "MACD", "Aroon Oscillator", "On-Balance Volume (OBV)"]
    tech_indicators = st.multiselect("Select the technical indicators you would like to include in the model", tech_indicator_options)

  if st.checkbox("Load model"):
  
    if model_name == "Recurrent Neural Network (RNN)":
    
      st.write("A recurrent neural network (RNN) is a type of artificial neural network which uses sequential data or time series data." + 
      " These deep learning algorithms are commonly used for ordinal or temporal problems, such as language translation, natural language processing (nlp)," + 
      "  speech recognition, and image captioning; they are incorporated into popular applications such as Siri, voice search, and Google Translate." + 
      " Like feedforward and convolutional neural networks (CNNs), recurrent neural networks utilize training data to learn." +
      " They are distinguished by their “memory” as they take information from prior inputs to influence the current input and output." + 
      " While traditional deep neural networks assume that inputs and outputs are independent of each other, the output of recurrent neural networks" + 
      " depend on the prior elements within the sequence. While future events would also be helpful in determining the output of a given sequence, unidirectional" + 
      " recurrent neural networks cannot account for these events in their predictions.")

      # Load model
      df = train_stock_prices
      df["Diff"] = df.Close.diff()
      df["y"] = df["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)

      df = append_TI(df, tech_indicators)

      df = df.drop(
          [ "Diff"],
          axis=1,
      ).dropna()


      X = StandardScaler().fit_transform(df.drop(["y"], axis=1))
      y = df["y"].values
      X_train, X_test, y_train, y_test = train_test_split(
          X,
          y,
          test_size=0.2,
          shuffle=False,
      )
      model = Sequential()
      model.add(SimpleRNN(2, input_shape=(X_train.shape[1], 1)))
      model.add(Dense(1, activation="sigmoid"))
      model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
      history = model.fit(X_train[:, :, np.newaxis], y_train, validation_split=0.2, epochs=100)
      y_pred = model.predict(X_test[:, :, np.newaxis])


      st.success("Success!")
      st.write(df)

      with st.form("form"):
        acc_checkbox = st.checkbox("Test Simple RNN Accuracy")
        if acc_checkbox:
          st.write("Model accuracy on test dataset: ", accuracy_score(y_test, y_pred > 0.5))

        loss_checkbox = st.checkbox("View Loss Plot")
        if loss_checkbox:
          plt.plot(history.history['loss'])
          plt.plot(history.history['val_loss'])
          plt.title('model loss')
          plt.ylabel('loss')
          plt.xlabel('epoch')
          plt.legend(['train', 'test'], loc='upper left')
          st.pyplot(plt)
          #st.write(history.history['acc'])

        visual_checkbox = st.checkbox("Visualize RNN")
        if visual_checkbox:
          plot_model(model, to_file='rnn_model_plot.png', show_shapes=True, show_layer_names=True)
          image = Image.open('rnn_model_plot.png')
          st.image(image)

        pred_checkbox = st.checkbox("Predict Tomorrows Movement")
        if pred_checkbox:
          y_pred = model.predict(X_test[:, :, np.newaxis][-1].reshape(1, -1))
          st.write(y_pred)
          #y_pred = model.predict(X_test[-1].reshape(1, -1))

          #if y_pred == 1:
           # st.write("The model predicts " + ticker_name + " stock will increase tomorrow")

          #else:
           # st.write("The model predicts " + ticker_name + " stock will decrease tomorrow")

        submitted = st.form_submit_button("Submit")
        if submitted:
          st.write("checkbox", loss_checkbox, "checkbox", acc_checkbox, "checkbox", visual_checkbox)

      # Display predictions and Cost history


    if model_name == 'XGBoost':

      st.write("As the name may reveal, XGBoost the Algorithm is a gradient boosting algorithm, a common technique in ensemble learning."+
              " To unpack that new phrase, ensemble learning is a type of machine learning that enlists many models to make predictions together."+ 
              " Boosting algorithms are distinguished from other ensemble learning techniques by building a sequence of initially weak models into increasingly more powerful models."+ 
              " Gradient boosting algorithms choose how to build a more powerful model using the gradient of a loss function that captures the performance of a model."+
              " Gradient boosting is a foundational approach to many machine learning algorithms."+
              " XGBoost has solidified its name in the boosting game with its use in many competition-winning models and prolific reference in research.")

      
      # Load model


      # Data Pre Processing


      # Train models


      # Display predictions and Cost history


    if model_name == 'Random Forest Classifier':

      st.write("""Random Forest (RF) is an ensemble learning procedure for regression and classification. As with XGBoost, RF employs many models to make predictions together.
                  The models utilized in RF are decision trees, and is implemented as an ensemble in order to reduce variance. Decision tress can have a tendency to run deep and overfit models,
                  thus random forests work well to average a multitude of decision trees and achieve an ideal result. RF is a great model for stock prediction, as it handles large datasets
                  well and predicts with relative speed. Hyperparameter tuning must be done carefully, as deep RF models can be difficult to interpret. """)

      
      # Load model
      df = train_stock_prices
      df["Diff"] = df.Close.diff()
      df["y"] = df["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)

      df = append_TI(df, tech_indicators)
            
        
      df = df.drop(
        ["Diff"],
        axis=1,
      ).dropna()
      X = df.drop(["y"], axis=1).values
      y = df["y"].values
      X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        shuffle=False,
      )

      @st.cache
      def Classify_RF(df):
        RF_Model = RandomForestClassifier()
        RF_Model.fit(
          X_train,
          y_train,
        )

        return RF_Model

      @st.cache
      def RF_Predictions(RF_Model):
        y_pred = RF_Model.predict(X_test)
        return y_pred


      RF_Model = Classify_RF(df)
      y_pred = RF_Predictions(RF_Model)
      

      st.success("Success!")
      st.write(df)
  
      with st.form("form"):
        feat_check = st.checkbox("See Feature Importance")
        if feat_check:

          features = ["Open", "High", "Low", "Close", "Adj Close", "Volume"]

          for indicator in tech_indicators:
            features.append(indicator)

          fig = go.Figure([go.Bar(x = features, y = RF_Model.feature_importances_)])
          fig.update_layout(title= 'Feature Importances', xaxis_title = 'Features')
          st.write(fig)

        acc_checkbox = st.checkbox("Test Random Forest Classifier Accuracy")
        if acc_checkbox:
          st.write("Model accuracy on test dataset: ", accuracy_score(y_test, y_pred))

        if st.checkbox("View Confusion Matrix"):
          st.write(confusion_matrix(y_test, y_pred))

        report_checkbox = st.checkbox("View Classification Report")
        if report_checkbox:
          report = classification_report(y_test, y_pred, output_dict=True)
          report = pd.DataFrame(report).transpose()
          st.write(report)

        pred_checkbox = st.checkbox("Predict Tomorrows Movement")
        if pred_checkbox:
          y_pred = RF_Model.predict(X_test[-1].reshape(1, -1))

          if y_pred == 1:
            st.write("The model predicts " + ticker_name + " stock will increase tomorrow")

          else:
            st.write("The model predicts " + ticker_name + " stock will decrease tomorrow")


        submitted = st.form_submit_button("Submit")
        if submitted:
          st.write("checkbox", feat_check, "checkbox", acc_checkbox, "checkbox", report_checkbox, "checkbox", pred_checkbox)


      # Data Pre Processing


      # Train models


      # Display predictions and Cost history

    if model_name == 'LSTM':
    
      df = train_stock_prices
      df["Diff"] = df.Close.diff()
      df["y"] = df["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)

      df = append_TI(df, tech_indicators)
      
      df = df.drop(
        ["Diff"],
        axis=1,
      ).dropna()

      X = StandardScaler().fit_transform(df.drop(["y"], axis=1))
      y = df["y"].values
      X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        shuffle=False,
      )
      model = Sequential()
      model.add(LSTM(2, input_shape=(X_train.shape[1], 1)))
      model.add(Dense(1, activation="sigmoid"))
      model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
      history = model.fit(X_train[:, :, np.newaxis], y_train, validation_split=0.2, epochs=100)
      y_pred = model.predict(X_test[:, :, np.newaxis])
      
      st.success("Success!")
      st.write(df)

      with st.form("form"):

        acc_checkbox = st.checkbox("Test LSTM Accuracy")
        if acc_checkbox:
          st.write("Model accuracy on test dataset: ", accuracy_score(y_test, y_pred > 0.5))

        loss_checkbox = st.checkbox("View Loss Plot")
        if loss_checkbox:
          plt.plot(history.history['loss'])
          plt.plot(history.history['val_loss'])
          plt.title('model loss')
          plt.ylabel('loss')
          plt.xlabel('epoch')
          plt.legend(['train', 'test'], loc='upper left')
          st.pyplot(plt)
          #st.write(history.history['acc'])

        visual_checkbox = st.checkbox("Visualize LSTM Network")
        if visual_checkbox:
          plot_model(model, to_file='lstm_model_plot.png', show_shapes=True, show_layer_names=True)
          image = Image.open('lstm_model_plot.png')
          st.image(image)

        submitted = st.form_submit_button("Submit")
        if submitted:
          st.write("checkbox", acc_checkbox, "checkbox", loss_checkbox, "checkbox", visual_checkbox)

      # As of now, Keras doesn't provide a way to extract feature importance. This is the closest I have come but
      # It does not work with sequential models
      if st.checkbox("Show Feature Importances"):
        import shap
        shap.initjs()
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_train[:, :, np.newaxis])

        # visualize the first prediction's explanation 
        shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:], matplotlib = True)

        st.pyplot(shap.summary_plot(shap_values, X_train[:, :, np.newaxis], plot_type="bar"))

######### PORTFOLIO OPTIMIZATION ##########

if st.checkbox("Run Monte Carlo Portfolio Optimization algorithm"):

    tickerOptions = ['AAPL', 'ROKU', 'MGNI', 'TSLA', 'AMD', 'PFE', 'MRNA', 'JNJ', "GOOGL","FB", "JPM", "BAC", "HSBC"]

    tickers = st.multiselect("Select the tickers you would like to consider in your portfolio", tickerOptions)

    # run optimization algorithm

    start_dateOP = st.date_input('Analyze data from', today)
    end_dateOP = st.date_input('Analyze data to', tomorrow)

    prices_df = pd.DataFrame()

    if st.checkbox("Run!"):

        for ticker in tickers:
            
            data = get_hist_data(ticker, start_dateOP, end_dateOP)

            data_date = data.reset_index()

            data_date[f"{ticker}"] = data_date['Adj Close']

            data_date = data_date.set_index('Date')
            
            if len(prices_df) < 1:
                prices_df = pd.concat([prices_df, data_date[f"{ticker}"]], axis=1)
            
            else:
                prices_df = prices_df.join(data_date[f"{ticker}"])

        st.write(prices_df)
        # "Optimize with Mean-Variance Optimization", 
        option = st.selectbox("Choose model", ["Monte Carlo Simulation"])

        # if option == "Optimize with Mean-Variance Optimization":

        #     # MAKE THIS USER INTERACTIVE
        #     mu = ema_historical_return(prices_df)
        #     S = exp_cov(prices_df)
        #     ef = EfficientFrontier(mu, S, solver="SCS")

        #     weights = ef.min_volatility()
        #     cleaned_weights = ef.clean_weights()
        #     port_performance = ef.portfolio_performance(verbose=True)
        #     st.write(port_performance)

        #     latest_prices = get_latest_prices(prices_df)
        #     da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)
        #     allocation, leftover = da.greedy_portfolio()
        #     st.write("Discrete allocation:", allocation)
        #     st.write("Funds remaining: ${:.2f}".format(leftover))

        if option == "Monte Carlo Simulation":

            np.random.seed(42) # for replicability

            returns_portfolio = prices_df.pct_change()
            st.write("Returns", returns_portfolio)
            cov = returns_portfolio.cov()

            portfolios = pd.DataFrame(columns=[*tickers, "Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"])

            port_returns = []
            port_volatility = []
            port_weights = []

            num_assets = len(prices_df.columns)
            num_portfolios = st.number_input("Insert number of portfolios to simulate", min_value=100, max_value=1000000)

            #make date a column so we can edit it
            prices_df = prices_df.reset_index()

            #change date column (timestamp) to date
            prices_df['index'] = pd.to_datetime(prices_df['index'])

            #set date back to index
            prices_df = prices_df.set_index('index')

            daily_indv_returns = prices_df.pct_change()
            daily_indv_returns = daily_indv_returns.dropna()

            start_time = time.time()

            for port in range(int(num_portfolios)):

                weights = np.random.random(num_assets)
                weights = weights / np.sum(weights) # Sum must equal 1

                portfolios.loc[port, tickers] = weights
                metrics = portfolio_metrics(returns_portfolio, weights, 0.02, port)
                portfolios.loc[port, ["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]] = \
                metrics.loc[port,["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]]
            
            end_time = time.time()

            st.write("It took ", round(end_time - start_time, 2), " seconds to simulate ", num_portfolios, " for ", (end_dateOP - start_dateOP).days, " days.")
            st.write(portfolios)

            # plot efficient frontier
            make_efficient_frontier(portfolios["Portfolio Variance"], portfolios["Expected Return"])

            if st.checkbox("Minimize volatility"):

                # Use new formula for better results
                optimal_min_portfolio = portfolios[portfolios["Portfolio Variance"]==portfolios["Portfolio Variance"].min()].T
                opt_min_weights = portfolios[portfolios["Portfolio Variance"]==portfolios["Portfolio Variance"].min()].to_numpy()[0][0:len(tickers)]
                min_allocation = optimal_portfolio_weights(opt_min_weights, tickers)
                st.write("Reccomended allocation for minimum variance", min_allocation)

                make_efficient_frontier_opt(portfolios["Portfolio Variance"], portfolios["Expected Return"], (optimal_min_portfolio.T)["Portfolio Variance"].values[0], (optimal_min_portfolio.T)["Expected Return"].values[0], 0, 0)

            if st.checkbox("Maximize Sharpe Ratio"):

                risk_factor = 0.01 # it is usually the treasury bond yield

                # Use new formula for better results
                optimal_risk_port = portfolios[portfolios["Sharpe Ratio"]==portfolios["Sharpe Ratio"].max()].T
                #optimal_risk_port_df = optimal_risk_port.to_frame()
                optimal_risk_port.columns = ['Summary']
                

                opt_weights = portfolios[portfolios["Sharpe Ratio"]==portfolios["Sharpe Ratio"].max()].to_numpy()[0][0:len(tickers)]
                allocation = optimal_portfolio_weights(opt_weights, tickers)
                st.write("Reccomended allocation for maximum Sharpe Ratio", allocation)

                make_efficient_frontier_opt(portfolios["Portfolio Variance"], portfolios["Expected Return"], 0, 0, (optimal_risk_port.T)["Portfolio Variance"][0], (optimal_risk_port.T)["Expected Return"][0], min_volatility = False)
        

Overwriting app.py


## Server set up

In [5]:
## Authtoken needed to connect to server

!ngrok authtoken 267bF5z4JLHYkOC3tXZj4e2GYW5_7hYxKXE8fRwrYw1niAjUE

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## Often, after running ngrok.connect(...) the tunnel fails to connect. If this happens simply close the window, run the cell ngrok.kill() and re run ngrok.connect(...). The issue will be fixed.

In [8]:
ngrok.kill()

In [9]:
!nohup streamlit run app.py &
url = ngrok.connect(port = '8501')
print(url)

nohup: appending output to 'nohup.out'
http://00f8-35-245-211-201.ngrok.io
